# 1. Import Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import warnings

# 2. Import Data

In [5]:
df = pd.read_csv("data/StudentsPerformance.csv")

In [8]:
df.columns = df.columns.str.replace(" ", "_")

In [6]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


# 3. Preparing X and Y variables

In [9]:
X = df.drop(columns=["math_score"], axis=1)
y = df["math_score"]

## Create a column transformer with 3 types of transformers

In [21]:
numerical_features = X.select_dtypes(exclude=["object"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

In [22]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, categorical_features),
        ("StandardScaler", numeric_transformer, numerical_features)
    ]
)

In [24]:
X = preprocessor.fit_transform(X)

# Train Test Split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
X_train.shape

(800, 19)

# Create and evaluate function to give all metrics after model training

In [30]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [31]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K Neighbors Regressor": KNeighborsRegressor(),
    "Descision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoostRegressor": CatBoostRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor()
}

In [32]:
model_list = []
r2_list = []

In [34]:
for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate the train and test datasets
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print("Model performance for training set.")
    print(" - Root mean squared error: {:.4f}".format(model_train_rmse))
    print(" - Mean absolute error: {:.4f}".format(model_train_mse))
    print(" - R2 Score: {:.4f}".format(model_train_r2))
    
    print("-"*35)
    
    print("Model performance for test set.")
    print(" - Root mean squared error: {:.4f}".format(model_test_rmse))
    print(" - Mean absolute error: {:.4f}".format(model_test_mse))
    print(" - R2 Score: {:.4f}".format(model_test_r2))
    
    r2_list.append(model_test_r2)
    print("="*70)
    print("\n")

    

Linear Regression
Model performance for training set.
 - Root mean squared error: 5.3257
 - Mean absolute error: 28.3633
 - R2 Score: 0.8742
-----------------------------------
Model performance for test set.
 - Root mean squared error: 5.4212
 - Mean absolute error: 29.3899
 - R2 Score: 0.8792


Lasso
Model performance for training set.
 - Root mean squared error: 6.5938
 - Mean absolute error: 43.4784
 - R2 Score: 0.8071
-----------------------------------
Model performance for test set.
 - Root mean squared error: 6.5197
 - Mean absolute error: 42.5064
 - R2 Score: 0.8253


Ridge
Model performance for training set.
 - Root mean squared error: 5.3233
 - Mean absolute error: 28.3378
 - R2 Score: 0.8743
-----------------------------------
Model performance for test set.
 - Root mean squared error: 5.3904
 - Mean absolute error: 29.0563
 - R2 Score: 0.8806


K Neighbors Regressor
Model performance for training set.
 - Root mean squared error: 5.7079
 - Mean absolute error: 32.5799
 - R2

In [40]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=["model_name", "r2_score"]).sort_values(by="r2_score", ascending=False)

,model_name,r2_score
11,Ridge,0.880593
2,Ridge,0.880593
0,Linear Regression,0.879222
9,Linear Regression,0.879222
14,Random Forest,0.853438
5,Random Forest,0.851752
7,CatBoostRegressor,0.851632
16,CatBoostRegressor,0.851632
8,AdaBoostRegressor,0.851333
17,AdaBoostRegressor,0.847250


In [39]:
model_list

['Linear Regression',
 'Lasso',
 'Ridge',
 'K Neighbors Regressor',
 'Descision Tree',
 'Random Forest',
 'XGBRegressor',
 'CatBoostRegressor',
 'AdaBoostRegressor',
 'Linear Regression',
 'Lasso',
 'Ridge',
 'K Neighbors Regressor',
 'Descision Tree',
 'Random Forest',
 'XGBRegressor',
 'CatBoostRegressor',
 'AdaBoostRegressor']